<a href="https://colab.research.google.com/github/suwilanji-chipofya-hadat/luse-data-fetcher/blob/main/luse_financial_data_dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import io
import re
import enum
import requests
from bs4 import BeautifulSoup

In [60]:
class Month(enum.Enum):
	JANUARY = 1
	FEBRUARY = 2
	MARCH = 3
	APRIL = 4
	MAY = 5
	JUNE = 6
	JULY = 7
	AUGUST = 8
	SEPTEMBER = 9
	OCTOBER = 10
	NOVEMBER = 11
	DECEMBER = 12

In [61]:
res = requests.get("https://luse.co.zm/market-data/")
soup = BeautifulSoup(res.content, 'html.parser')

In [71]:
pdf_links = soup.find_all('a', href=re.compile(r'\.pdf$'))
links = [link['href'] for link in pdf_links]

In [74]:
data = []
for link in links:
  res = requests.get(link, stream=True)
  if res.status_code == 200:
    f_stream = io.BytesIO()
    for chunk in res.iter_content(chunk_size=8192):
      f_stream.write(chunk)
    f_stream.seek(0)
    data.append(f_stream)

In [82]:
l = links[0]
a = l.split('/')[-1]
al = a.split('-')
d = f"{al[0]} {al[1]} {al[2]}"
d

'30 June 2022'

In [94]:
import tabula
import pandas as pd

class Config:
    pages = 1
    relative_area = True
    relative_columns = True
    area = [30, 0, 894, 400]

def get_data(src_pdf, config=Config()) -> (bool, pd.DataFrame, str):
    try:
        data = tabula.read_pdf(src_pdf, pages=config.pages, relative_area=config.relative_area, relative_columns=config.relative_columns, area=config.area)
        df = data[0]

        # Drop rows with NaN values
        df = df.dropna(subset=df.columns, how='all')

        # Reset index after dropping rows
        df = df.reset_index(drop=True)

        # Fill NaN values with an empty string
        df = df.fillna("")

        # Combine header rows into a single row
        df.columns = [' '.join(filter(None, col)) for col in df.columns]

        return (True, df, "")
    except Exception as e:
        return (False, pd.DataFrame(), str(e))


In [ ]:
import tempfile
import pandas as pd
import os

dfs = []
for d in data:
  with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_file:
    temp_file.write(d.getvalue())
    src = temp_file.name
    df = get_data(src)
    dfs.append(df)
    temp_file.close()

In [117]:
new_dfs = []
for df in dfs:
  df[1].columns = ['instrument', 'bid_qty', 'bid_price', 'ask_price', 'ask_qty','last_trade_price', 'net_change', 'closing_price', 'total_turnover','average_price', 'last_traded_size', 'week_52_high', 'week_52_low','opening_price', 'change', 'previous_closing_price', 'total_trades','trade_volume', 'foreign_buys', 'foreign_sells']
  df_mod = df[1].iloc[2:-2]
  new_dfs.append(df_mod)

In [ ]:
!pip install sqlalchemy

In [125]:
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData, Table, Date

# Replace 'your_username', 'your_password', 'your_host', and 'your_database' with your actual PostgreSQL credentials
DATABASE_URL = ''
engine = create_engine(DATABASE_URL)

metadata = MetaData()

stock_data_table = Table(
    'stock_data',
    metadata,
    Column('date', Date),
    Column('instrument', String),
    Column('bid_qty', Integer),
    Column('bid_price', Float),
    Column('ask_price', Float),
    Column('ask_qty', Integer),
    Column('last_trade_price', Float),
    Column('net_change', Float),
    Column('closing_price', Float),
    Column('total_turnover', Float),
    Column('average_price', Float),
    Column('last_traded_size', Integer),
    Column('week_52_high', Float),
    Column('week_52_low', Float),
    Column('opening_price', Float),
    Column('change', Float),
    Column('previous_closing_price', Float),
    Column('total_trades', Integer),
    Column('trade_volume', Integer),
    Column('foreign_buys', Integer),
    Column('foreign_sells', Integer)
)

metadata.create_all(engine)


In [131]:
m_i = {
    "January": "01",
    "February": "02",
    "March": "03",
    "April": "04",
    "May": "05",
    "June": "06",
    "July": "07",
    "August": "08",
    "September": "09",
    "October": "10",
    "November": "11",
    "December": "12"
}

In [150]:
a = [d.split('/')[-1] for d in links]
a[142] = "13-December-2022-Trade-Summary-Report.pdf"

In [151]:

dates = []
for i, a in enumerate(a):
  an = a.split('-')
  s = f"{an[2]}-{m_i[an[1]]}-{an[0]}"
  dates.append(s)

In [ ]:
for i, df in enumerate(new_dfs):
  print(i)
  df['date'] = dates[i]

In [160]:
from sqlalchemy import create_engine
engine = create_engine('')  # Adjust connection details

for i, df in enumerate(new_dfs):
    df.to_sql(f'stock_data', engine, index=False, if_exists='append')


In [163]:
import sqlite3 as sql

conn = sql.connect("dump.sqlite3")
cursor = conn.cursor()
query = '''
CREATE TABLE stock_data (
    date DATE,
    instrument TEXT,
    bid_qty INTEGER,
    bid_price REAL,
    ask_price REAL,
    ask_qty INTEGER,
    last_trade_price REAL,
    net_change REAL,
    closing_price REAL,
    total_turnover REAL,
    average_price REAL,
    last_traded_size INTEGER,
    week_52_high REAL,
    week_52_low REAL,
    opening_price REAL,
    change REAL,
    previous_closing_price REAL,
    total_trades INTEGER,
    trade_volume INTEGER,
    foreign_buys INTEGER,
    foreign_sells INTEGER
);
'''
cursor.execute(query)
conn.commit()


In [164]:
for i, df in enumerate(new_dfs):
    df.to_sql(f'stock_data', conn, index=False, if_exists='append')


In [166]:
conn.close()